# Example 2: Gibbs sampling for original posterior 

In [1]:
# To start this notebook with more than one thread run "export JULIA_NUM_THREADS=4" in the terminal 
# before starting the jupyter notebook

# Ensure that Julia was launched with an appropriate number of threads
println(Threads.nthreads())

6


In [2]:
# Import module. 
using Revise
using PriorNormalization

In [3]:
# Import packages 
using AdaptiveMCMC # for using adaptive MCMC sampling
using ApproxFun # for approximating gammainvccdf by a Chbychev interpolant  
using CairoMakie # for plots 
using Dates # to measure computational time 
using Distributions
using JLD2 # for saving and loading results
using FlexiMaps # for log-range  
using ForwardDiff # for AD
using LinearAlgebra # to represent the identity matrix as "I" 
using Random # for generating random noise 
using StatsBase # for defining customized distributions 
using StatsFuns # for defining customized distributions 
using StatsPlots # for plotting 
using SparseArrays # for efficient storing of the forward operator 
using SpecialFunctions
using Turing # for setting up the model and sampling 
using Optim # for ML and MAP estimation 
using Plots: Plots, plot, plot!, scatter, scatter!, savefig, surface, surface!
using ProgressMeter # to show progress 

In [4]:
# Prepare arguments.  
nr_chains = 6 # number of chains to sample 
nr_samples_raw = 10^3 # number of samples 
thin = 10^0 # Thinning factor; only every thin-th sample is stored
init = "MAP" # Initialization option: "MAP", "prior"
nr_samples = Int64( nr_samples_raw/thin )

# Tests: 
# nr_samples_raw = 10^3, thin = 10^0: 3.6s & 2.5s
# nr_samples_raw = 10^4, thin = 10^0: 25s & 48s
# nr_samples_raw = 10^5, thin = 10^1: 664 & 466s
# nr_samples_raw = 10^6, thin = 10^2: 
# nr_samples_raw = 10^7, thin = 10^3:  

1000

## Signal deblurring problem: Define the data model 

In [5]:
# Model parameters 
σ² = 0.03^2 # noise variance 
kernel_width = 0.02 # width of the Gaussian kernel
N_dense = 1_000 # number of points for the dense model
N_coarse = 128 # number of points for the coarse model
tt = [0.17, 0.39, 0.48, 0.73, 0.83] # Positions of the increments
dx = [1, -2.4, 2.8, -0.6, -0.8] # Values of the increments

5-element Vector{Float64}:
  1.0
 -2.4
  2.8
 -0.6
 -0.8

In [6]:
# Define the piecewise constant signal. 
function signal(t; tt=[0.17, 0.39, 0.48, 0.73, 0.83], 
    dx = [1, -2.4, 2.8, -0.6, -0.8])
    x = 0
    Ij = findall(x -> x < t, tt)
    if !isempty(Ij)
        x = sum(dx[Ij])
    end
    
    return x
end

signal (generic function with 1 method)

In [7]:
# Set up the dense data model  

# Generate the dense grid (we assume that the signal vanishes at t=0)
t_dense = (1:N_dense) / N_dense 

# Generate the dense forward operator 
S = reshape(repeat(t_dense, N_dense, 1), N_dense, N_dense)
T = S'
F_dense = (6.4/N_dense) * exp.(-1/(2*kernel_width^2) * (T.-S).^2) 

# Generate the dense step signal and observations 
x_dense = signal.(t_dense) # signal values 
y_dense = F_dense * x_dense # observations

1000-element Vector{Any}:
  3.74278937539227e-18
  5.744144243534786e-18
  8.79393927412621e-18
  1.3429811599511571e-17
  2.0459022030414613e-17
  3.1090554911693094e-17
  4.713037498942281e-17
  7.126926866908596e-17
  1.0750603997755944e-16
  1.6176805186960793e-16
  2.428187113838113e-16
  3.6358103971191115e-16
  5.430629429581705e-16
  ⋮
  2.653109939010615e-16
  1.693119370541826e-16
  1.0509846120628357e-16
  6.233097419208321e-17
  3.399747075949146e-17
  1.535468383152762e-17
  3.2021048354653674e-18
 -4.614439835542763e-18
 -9.541335133345305e-18
 -1.2547439067780209e-17
 -1.4280276203172293e-17
 -1.5171704985465577e-17

In [8]:
# Set up the coarse data model  

# Generate the dense grid (we assume that the signal vanishes at t=0)
t_coarse = (1:N_coarse) / N_coarse 

# Generate the dense forward operator 
S = reshape(repeat(t_coarse, N_coarse, 1), N_coarse, N_coarse)
T = S'
F_coarse = (6.4/N_coarse) * exp.(-1/(2*kernel_width^2) * (T.-S).^2) 

# Get the coarse grid and forward operator 
stride = 6 # use every stride-th point 
t_obs = t_coarse[1:stride:end]
F_coarse = F_coarse[1:stride:end, :]

# Find the nearest points in the dense grid
m = length(t_obs)
I_dense = zeros(Int, m)
for j in 1:m
    I_dense[j] = argmin(abs.(t_dense .- t_obs[j]))
end
     
# Coarse data with added noise
Random.seed!(123) # Setting the seed 
y_coarse = y_dense[I_dense] .+ sqrt(σ²)*randn(m);

In [9]:
# Invertible finite difference matrix
aux = ones(N_coarse) * [-1.0, 1.0]'
L = spzeros(Float64,N_coarse,N_coarse)
L[2:end,:] = spdiagm(0=>-1*ones(N_coarse), 1=>ones(N_coarse))[1:N_coarse-1,1:N_coarse]
L[1,1] = 1 

# Change coordinates to promot sparsity in z = Lx
FL = F_coarse / L # F_coarse * inv(L)

# Whitening
FL_w = (1/sqrt(σ²)) * FL
y_w = (1/sqrt(σ²)) * y_coarse

# Rename varables for simplicity 
F = FL_w 
y = y_w 
M, N = size(F)

(22, 128)

## Select the model: $r=-1$

In [10]:
# Select hyper-hyper-parameters 
model_nr = 4 

# Parameter of of generalized gamma hyper-prior 
r_range = [ 1.0, .5, -.5, -1.0 ]; 
β_range = [ 1.501, 3.0918, 2.0165, 1.0017 ];
ϑ_range = [ 5*10^(-2), 5.9323*10^(-3), 1.2583*10^(-3), 1.2308*10^(-4) ];

# Select hyper-hyper-parameters 
r = r_range[model_nr] # power parameter 
β = β_range[model_nr] # shape parameter 
ϑ = ϑ_range[model_nr] # scale parameter

0.00012308

## Initialization

In [11]:
# Load the MAP estimates 

# MAP estimate of the original posterior 
@load "data/deblurring_model$(model_nr)_MAP_original.jld2" θ_MAP z_MAP x_MAP
θ_MAP

128-element Vector{Float64}:
 4.997527572912228e-5
 5.000659064280895e-5
 5.003308921604457e-5
 5.004673545790628e-5
 5.004216155492188e-5
 5.0019355238489405e-5
 4.9983725274028313e-5
 4.994324263582611e-5
 4.990450263891553e-5
 4.987031070624744e-5
 4.984008721701541e-5
 4.981213130486383e-5
 4.9785761716268164e-5
 ⋮
 4.96922514167353e-5
 4.9692024722620115e-5
 4.969199501004405e-5
 4.969208013435234e-5
 4.9692415605844076e-5
 4.969307366154106e-5
 4.9693858305752975e-5
 4.9694375057087454e-5
 4.9694302754215614e-5
 4.969364991573615e-5
 4.969278712728878e-5
 4.969215803777444e-5

In [12]:
# Load the MAP estimates 

# MAP estimate of the original posterior 
@load "data/deblurring_model$(model_nr)_MAP_original.jld2" θ_MAP z_MAP x_MAP
# Initialize an empty vector to store the interleaved values
original_MAP = Vector{Float64}(undef, 2*N)
# Interleave τ_MAP and u_MAP
original_MAP[1:2:end-1] .= θ_MAP
original_MAP[2:2:end] .= z_MAP

# MAP estimate of the prior-normalized posterior 
@load "data/deblurring_model$(model_nr)_MAP_priorNormalized.jld2" τ_MAP u_MAP
# Initialize an empty vector to store the interleaved values
priorNormalized_MAP = Vector{Float64}(undef, 2*N)
# Interleave τ_MAP and u_MAP
priorNormalized_MAP[1:2:end-1] .= τ_MAP;
priorNormalized_MAP[2:2:end] .= u_MAP;

In [13]:
## Genrate random samples from the original prior 

original_prior = Array{Float64}(undef, 2*N, nr_chains)
# Generate θ-samples from the gamma distribution
θ_prior_samples = rand(Gamma(β,1), N, nr_chains)
# Transform them into samples of the generalized gamma distribution 
θ_prior_samples = ϑ * θ_prior_samples.^(1/r)

# Generate z-samples from the conditional Gaussian prior 
z_prior_samples = Array{Float64}(undef, N, nr_chains)
# Loop over each chain and each sample to generate the normal samples
for j in 1:nr_chains
    for n in 1:N
        # Standard deviation is sqrt(θ_prior_samples[i, j])
        σ = sqrt(θ_prior_samples[n, j])
        z_prior_samples[n, j] = rand(Normal(0, σ))
    end
    original_prior[1:2:end,j] .= θ_prior_samples[:,j]
    original_prior[2:2:end,j] .= z_prior_samples[:,j]
end

In [14]:
# Choose an initialization for the MCMC chains 
init_param_original = Array{Float64}(undef, 2*N, nr_chains)

# Use MAP estimate 
if init=="MAP"
    # Select the initial set of parameters 
    for j in 1:nr_chains 
        init_param_original[:,j] = original_MAP[:]
    end

    # Select the file names for saving the later MCMC results 
    # Original model 
    filename_original = joinpath(
        "data", 
        "deblurring_model$(model_nr)_mcmc_initMAP_Gibbs_original_samples$(nr_samples_raw)_thin$(thin)_chains$(nr_chains).jld2"
    )

# Use random prior samples 
elseif init=="prior"
    # Select the initial set of parameters 
    for j in 1:nr_chains 
        init_param_original[:,j] = original_prior[:,j]
    end

    # Select the file names for saving the later MCMC results 
    # Original model 
    filename_original = joinpath(
        "data", 
        "deblurring_model$(model_nr)_mcmc_initPrior_Gibbs_original_samples$(nr_samples_raw)_thin$(thin)_chains$(nr_chains).jld2"
    )

# Throw an error if none of the available options is provided
else
    error("Invalid initialization option provided: $init. Please choose either 'MAP' or 'prior'.")
end

"data/deblurring_model4_mcmc_initMAP_Gibbs_original_samples1000_thin1_chains6.jld2"

## Gibbs sampling

In [15]:
# Define the Gibbs sampling function
function gibbs_sampler(θ_init, iterations, F, y, r, β, ϑ)
    M, N = size(F) 
    Ft = transpose(F)
    FtF = Ft*F
    Fty = Ft*y
    
    # Initialize latent variables and parameters
    z = randn(N)  # Initial guess for u
    θ = θ_init

    # Preallocate space to store samples
    samples_z = zeros(iterations, N)
    samples_θ = zeros(iterations, N)

    for i in 1:iterations
        
        # Step 1: Sample z | y, θ
        v1 = randn(M)
        v2 = randn(N) 
        w = Ft*v1 + Diagonal( θ.^(-1/2) )*v2
        C = FtF + Diagonal( θ.^(-1) )
        rhs = Fty + w
        z = C\rhs

        # Step 2: Sample θ | y, z
        for n in 1:N
            θ[n] = rand( InverseGamma(β+0.5, ϑ+0.5*(z[n]^2) ) )
        end

        # Store samples
        samples_z[i, :] = z
        samples_θ[i, :] = θ
    end

    return samples_θ, samples_z
end

gibbs_sampler (generic function with 1 method)

In [16]:
# Initialize
nr_parameters = 2*N
Theta_init = zeros(Float64, N, nr_chains)
Theta = zeros(Float64, nr_samples_raw, N, nr_chains)
Z = zeros(Float64, nr_samples_raw, N, nr_chains)
chn_values = zeros(Float64, nr_samples, 2*N, nr_chains)

# Start the wall clock timer
wall_start = now()

# Use multiple threads. 
@showprogress Threads.@threads for j in 1:nr_chains   
    
    # Initilization 
    Theta_init[:,j] = init_param_original[1:2:end-1,j]
    
    # Generate samples
    Theta[:,:,j], Z[:,:,j] = gibbs_sampler(
        Theta_init[:,j], # initilization 
        nr_samples_raw, # number of samples 
        F, y, r, β, ϑ # data and prior model parameters 
    )
    
    # Store values 
    
    chn_values[:,1:2:end-1,j] = Theta[1:thin:nr_samples_raw,:,j]
    chn_values[:,2:2:end,j] = Z[1:thin:nr_samples_raw,:,j]
    
    # Clear memory after each chain
    GC.gc()
end

# End the wall clock timer
wall_end = now()
wall_duration_ms = wall_end - wall_start
# Convert wall duration to seconds
wall_duration_original = Dates.value(wall_duration_ms) / 1000

# Define the parameter names (θ[1], z[1], θ[2], z[2], ...)
param_names_θ = [string("θ[", i, "]") for i in 1:N ]
param_names_z = [string("z[", i, "]") for i in 1:N ]

# Interleave θ and z names
param_names = Vector{String}(undef, nr_parameters)
param_names[1:2:end-1] .= param_names_θ
param_names[2:2:end] .= param_names_z

# Create the Chains object
chn_original = Chains(chn_values, Symbol.(param_names));

# Free the memory occupied by chn_values
chn_values = nothing
GC.gc()  # Optionally trigger garbage collection manually

Progress: 100%|█████████████████████████████████████████| Time: 0:00:07


In [18]:
# Save the MCMC chain and computational time
@save filename_original chn_original wall_duration_original

# Multivariate potential scale reduction factor (MPSRF) 
# To check convergence: Should be below 1.1

gelmandiag_multivariate(chn_original)

(Gelman, Rubin, and Brooks diagnostic (256 x 3), 5.824644507705167)